Skyline probleme model

Datas :
* m is number of buildings
* n is the number of clients
* p_i numbers of floor occuped by the client i
* g_i benefits given by the client i
* K the max floors of each buildings
* P_j price of floor j

In [6]:
# Librarys
using JuMP
using GLPK

const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;


In [7]:
#main function

function skyline_problem(buildings_number,clients_number
        ,clients_floors,client_gains,building_max_size,floor_prices)


    # Creation of the model
    model = Model(with_optimizer(GLPK.Optimizer))

    # Creation of variables
    
    # if the client i is in the building l, A[i,l] = 1 , else A[i,l] = 0
    @variable(model,A[1:clients_number,1:buildings_number],Bin)
    
    # if the floor k of the building l is used, B[k,l] = 1, else B[k,l] = 0
    @variable(model,B[1:building_max_size,1:buildings_number],Bin) 
    #variable that will countain the number of floor used on each buildings (integer)
    @variable(model,0<=B_Bis[1:buildings_number]<=building_max_size,Int)
    #-B_Bis makes the link between the two other
    
    # objective function
    @objective(model, Max, 
        sum(clients_gains[i].*A[i,l] for i in 1:clients_number for l in 1:buildings_number) #sum of benefits
        -
        sum(floor_prices[k].*B[k,l] for k in 1:building_max_size for l in 1:buildings_number)) #sum of buildings price

    #Constraints

    #Each client can be in only one building
    for i in 1:clients_number
        @constraint(model,sum(A[i,:]) <= 1)
    end
    
    #Constraints on the size of buildings
    
    for l in 1:buildings_number
        #link B_Bis, the size of buildings, and B, attribution of floors
        @constraint(model,B_Bis[l]==sum(B[k,l] for k in 1:building_max_size))
        #we can't build a floor if the floor under isn't built
        for k in 1:building_max_size-1
            @constraint(model,B[k,l]>=B[k+1,l])
        end
        #link B_Bis, the size of buildings and the askings of clients
        @constraint(model,B_Bis[l]==sum(clients_floors[i].*A[i,l] for i in 1:clients_number))
    end

    #Resolving
    optimize!(model)

    status = termination_status(model)

    if status == INFEASIBLE
        println("The problem isn't resolvable")
    elseif status == UNBOUNDED
        println("the probleme doesn't have borns")
    elseif status == OPTIMAL
        println("Optimum = ", JuMP.objective_value(model))
        println("Solution :")
        println("A = ",JuMP.value.(A))
        println("B = ",JuMP.value.(B))
        println("B_Bis = ",JuMP.value.(B_Bis))
        
        println("Show :")
        for l in 1:buildings_number
            println("\t-The building ",l," has ",JuMP.value(B_Bis[l])," floors.")
            cout = sum(floor_prices[k].*JuMP.value(B[k,l]) for k in 1:building_max_size)
            println("\tIt costs ",cout,"€ to build.")
            for i in 1:clients_number 
                if JuMP.value(A[i,l])==1
                    println("\t\tClient ",i," is in the building ",l)
                end
            end
            gain = sum(clients_gains[i].*JuMP.value(A[i,l]) for i in 1:clients_number)
            println("\tThe clients benefits are ",gain,"€.")
            println("\tThe building ",l," give ",gain-cout," €.")    
            println()
        end
        
        println("The benefits of the project are ", JuMP.objective_value(model),"€.")
        println("drawinf of solution:")
        println()
        for l in 1:buildings_number
            print(l,"\t")
        end
        println()
        for k in 1:building_max_size
            for l in 1:buildings_number
                if JuMP.value(B[building_max_size-k+1,l])==0
                    print(".\t")
                else
                    print("M\t")
                end
            end
            println()
        end
    end
end

skyline_problem (generic function with 1 method)

In [8]:
#création d'une instance test du jeu:
#nombre d'immeubles
buildings_number = 3
#nombre de clients
clients_number = 4
#nombre d'étages pour chaque demande des clients
clients_floors = [1 1 3 4]
#gains pour chaque demande des clients
clients_gains = [5 8 100 300]
#taille maximal d'un immeuble = somme des étages de toutes les demandes
building_max_size = sum(clients_floors)
#prix de construction de chaque étage d'un immeuble
#pourra être créé via une fonction
floor_prices = [10 20 40 80 160 320 640 1280 2560] 

skyline_problem(buildings_number,clients_number
        ,clients_floors,clients_gains,building_max_size,floor_prices)

Optimum = 180.0
Solution :
A = [0.0 0.0 0.0; 0.0 0.0 0.0; 1.0 0.0 0.0; 0.0 0.0 1.0]
B = [1.0 0.0 1.0; 1.0 0.0 1.0; 1.0 0.0 1.0; 0.0 0.0 1.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
B_Bis = [3.0, 0.0, 4.0]
Show :
	-The building 1 has 3.0 floors.
	It costs 70.0€ to build.
		Client 3 is in the building 1
	The clients benefits are 100.0€.
	The building 1 give 30.0 €.

	-The building 2 has 0.0 floors.
	It costs 0.0€ to build.
	The clients benefits are 0.0€.
	The building 2 give 0.0 €.

	-The building 3 has 4.0 floors.
	It costs 150.0€ to build.
		Client 4 is in the building 3
	The clients benefits are 300.0€.
	The building 3 give 150.0 €.

The benefits of the project are 180.0€.
drawinf of solution:

1	2	3	
.	.	.	
.	.	.	
.	.	.	
.	.	.	
.	.	.	
.	.	M	
M	.	M	
M	.	M	
M	.	M	


In [9]:
#algorithm dynamic programmtion

#remarque: items commencé à 1 et non à 0
function skyline_problem_dynamic_resolution(clients_number,
        clients_floors,client_gains,building_max_size)
    V = zeros(Int64,building_max_size+1,clients_number)

    for k in 1:building_max_size+1 
        if  k > clients_floors[1]
            V[k,1] = clients_gains[1]
        end
    end
    for k in 1:building_max_size+1     
        for i in 2:clients_number
            if k <= clients_floors[i]
                V[k,i] = V[k,i-1]
            else
                V[k,i] = max(V[k,i-1],clients_gains[i]+V[k-clients_floors[i],i-1])
            end
        end
    end
    print("\t")
    for i in 1:clients_number
            print(i,"\t")
    end
    println()
    for k in 1:building_max_size+1
        print(k-1,"\t")
        for i in 1:clients_number
            print(V[k,i],"\t")
        end
        println()
    end
    println("PATH:")
    k = building_max_size+1
    for invi in 1:clients_number-1
        i = clients_number - invi +1
        println("(k,i,V[k,i]) = (",k-1,",",i,",",V[k,i],")")
        if V[k,i] != V[k,i-1]
            k = k - clients_floors[i]
        end
    end
    println("(k,i,V[k,i]) = (",k-1,",",1,",",V[k,1],")")
end




skyline_problem_dynamic_resolution (generic function with 1 method)

In [10]:

#resolution d'un sac à dos, exo3 TD7 d'OC pour vérification

clients_number = 5
clients_floors = [7 6 14 7 9]
clients_gains = [1 2 5 6 11]
building_max_size = sum(clients_floors)

skyline_problem_dynamic_resolution(clients_number,
        clients_floors,client_gains,15)


LoadError: UndefVarError: client_gains not defined

In [11]:
buildings_number = 2
clients_number = 5
clients_floors = [3 3 3 2 2]
clients_gains = [350 300 325 225 225]
building_max_size = sum(clients_floors)
floor_prices = [80 80 100 100 120 120 140 140 160 160 180 180 200]
skyline_problem(buildings_number,clients_number ,clients_floors,clients_gains,building_max_size,floor_prices)


Optimum = 180.0
Solution :
A = [1.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0]
B = [1.0 1.0; 1.0 1.0; 1.0 1.0; 0.0 1.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]
B_Bis = [3.0, 4.0]
Show :
	-The building 1 has 3.0 floors.
	It costs 260.0€ to build.
		Client 1 is in the building 1
	The clients benefits are 350.0€.
	The building 1 give 90.0 €.

	-The building 2 has 4.0 floors.
	It costs 360.0€ to build.
		Client 4 is in the building 2
		Client 5 is in the building 2
	The clients benefits are 450.0€.
	The building 2 give 90.0 €.

The benefits of the project are 180.0€.
drawinf of solution:

1	2	
.	.	
.	.	
.	.	
.	.	
.	.	
.	.	
.	.	
.	.	
.	.	
.	M	
M	M	
M	M	
M	M	
